# Training and Validation

In [39]:
import pandas as pd

df = pd.read_csv("../data/classifier/concat.csv")
df

,id,text,label,num_words,clean_text
0,1576050330114363392,@joicehasselmann Bolsonaro NÃO quer aproveitad...,-1,7,bolsonaro não quer aproveitadores como você.
1,1576050330110160901,o bolsonaro é um anjo enviado de deus para nos...,1,22,o bolsonaro é um anjo enviado de deus para nos...
2,1576050318852640769,@AndradeRNegro2 @Loucosdovalle Bolsonaro é con...,1,28,bolsonaro é contra o aborto e defende a vida...
3,1576050318596399104,@Victorrmsk É a milésima vez que ele fala ao v...,0,15,é a milésima vez que ele fala ao vivo que não...
4,1576050309822287873,"@OGloboPolitica Bolsonaro é tão fake, q até o ...",-1,18,"bolsonaro é tão fake, q até o laranja q ele l..."
...,...,...,...,...,...
697,1597340353430171648,Hahaha e os patriotas na chuva \nhttps://t.co/...,-1,7,hahaha e os patriotas na chuva \n
698,1597340352733904897,@PATRlOTAS Esse idiota @BolsonaroSP não se ele...,-1,34,"esse idiota não se elege nunca mais, o brasi..."
699,1597340328813506560,@rafaelgloves Depois disso se o pai dele nao d...,-1,28,depois disso se o pai dele nao der o contra g...
700,1597340303308259329,@ducavendish Modinha de cretinos. Bolsonaro é ...,1,51,modinha de cretinos. bolsonaro é o único resp...


In [40]:
import numpy as np

inputs = df['clean_text'].to_numpy()
target = df['label'].to_numpy()

In [41]:
from sentence_transformers import SentenceTransformer

model_transform = SentenceTransformer('../distiluse-base-multilingual-cased-v1')
model_transform

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [42]:
embeddings = model_transform.encode(inputs)

In [43]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(embeddings, target, test_size=0.2, stratify = target, random_state=42)

In [44]:
#from imblearn.over_sampling import SMOTE 

#sm = SMOTE(random_state=42)
#x_train, y_train = sm.fit_resample(x_train, y_train)

# KNN

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)

param_grid = {'n_neighbors' : list(range(2, 40)), 'weights' : ['distance'], 'metric': ['cosine']}

clf = GridSearchCV(estimator = KNeighborsClassifier(n_jobs = -1), param_grid = param_grid, cv = skf, n_jobs=-1)
clf.fit(x_train, y_train)
df_results = pd.DataFrame(clf.cv_results_)

In [46]:
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002000,6.323345e-04,0.037609,0.006565,cosine,2,distance,"{'metric': 'cosine', 'n_neighbors': 2, 'weight...",0.486726,0.535714,0.526786,0.553571,0.580357,0.536631,0.030943,38
1,0.002201,3.995922e-04,0.041599,0.004962,cosine,3,distance,"{'metric': 'cosine', 'n_neighbors': 3, 'weight...",0.522124,0.616071,0.633929,0.651786,0.616071,0.607996,0.044932,32
2,0.002001,6.334658e-04,0.036799,0.004664,cosine,4,distance,"{'metric': 'cosine', 'n_neighbors': 4, 'weight...",0.548673,0.616071,0.642857,0.642857,0.580357,0.606163,0.036776,34
3,0.002704,1.164909e-03,0.038519,0.007210,cosine,5,distance,"{'metric': 'cosine', 'n_neighbors': 5, 'weight...",0.566372,0.589286,0.625000,0.633929,0.625000,0.607917,0.025820,33
4,0.002001,1.441595e-06,0.036400,0.001498,cosine,6,distance,"{'metric': 'cosine', 'n_neighbors': 6, 'weight...",0.566372,0.562500,0.598214,0.589286,0.598214,0.582917,0.015487,37
5,0.001801,7.477569e-04,0.036600,0.004409,cosine,7,distance,"{'metric': 'cosine', 'n_neighbors': 7, 'weight...",0.575221,0.616071,0.642857,0.669643,0.607143,0.622187,0.032090,13
6,0.001800,4.004003e-04,0.038802,0.003430,cosine,8,distance,"{'metric': 'cosine', 'n_neighbors': 8, 'weight...",0.584071,0.633929,0.625000,0.660714,0.625000,0.625743,0.024608,8
7,0.002800,2.638171e-03,0.033601,0.006470,cosine,9,distance,"{'metric': 'cosine', 'n_neighbors': 9, 'weight...",0.592920,0.642857,0.651786,0.633929,0.607143,0.625727,0.022187,9
8,0.002001,1.094365e-03,0.030800,0.001327,cosine,10,distance,"{'metric': 'cosine', 'n_neighbors': 10, 'weigh...",0.592920,0.625000,0.625000,0.625000,0.625000,0.618584,0.012832,19
9,0.001600,4.899410e-04,0.035999,0.006326,cosine,11,distance,"{'metric': 'cosine', 'n_neighbors': 11, 'weigh...",0.601770,0.633929,0.651786,0.660714,0.633929,0.636425,0.020196,1


In [47]:
model_knn = clf.best_estimator_
model_knn

KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=11,
                     weights='distance')

In [48]:
y_pred_knn = model_knn.predict(x_val)
y_pred_knn

array([ 0, -1,  0, -1, -1,  0,  1, -1, -1,  0,  1,  0,  0, -1,  0,  0,  0,
        1, -1, -1,  0, -1, -1, -1,  0,  0,  0,  0, -1, -1, -1,  0,  0, -1,
       -1,  0, -1,  0, -1, -1,  0, -1,  0,  1, -1, -1,  0,  0,  0, -1, -1,
       -1, -1,  0, -1, -1,  0,  1, -1,  0, -1,  0, -1, -1, -1,  1,  0, -1,
        0,  1,  0, -1, -1,  0, -1,  0,  0, -1, -1,  0, -1,  0,  0, -1, -1,
       -1,  0,  0,  0,  0, -1,  0, -1,  1,  0, -1, -1, -1, -1,  0, -1, -1,
       -1, -1,  0, -1,  0,  0, -1,  0,  0,  0,  1,  0, -1, -1,  0,  0, -1,
        0, -1, -1, -1, -1,  0,  0,  1, -1, -1,  0, -1, -1, -1,  1,  0,  0,
       -1, -1,  0,  0, -1], dtype=int64)

In [49]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred_knn))

              precision    recall  f1-score   support

          -1       0.63      0.72      0.67        61
           0       0.67      0.68      0.67        59
           1       0.55      0.29      0.37        21

    accuracy                           0.64       141
   macro avg       0.61      0.56      0.57       141
weighted avg       0.63      0.64      0.63       141



# VotingClassifier

In [50]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

#model_vc = VotingClassifier(estimators=[('ABC', AdaBoostClassifier(n_estimators=500)), ('GBC', GradientBoostingClassifier(n_estimators=500)), ('MLP', MLPClassifier(random_state=1, #max_iter=500))], voting='hard')

param_grid = {'n_estimators' : [3000], 'learning_rate' : [0.5, 1.0, 1.5]}

model_vc = AdaBoostClassifier(random_state = 42)

clf = GridSearchCV(estimator = model_vc, param_grid = param_grid, cv = skf, n_jobs=-1)
clf.fit(x_train, y_train)
df_results = pd.DataFrame(clf.cv_results_)


In [51]:
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,83.187063,1.149312,0.621315,0.022104,0.5,3000,"{'learning_rate': 0.5, 'n_estimators': 3000}",0.548673,0.633929,0.678571,0.642857,0.625000,0.625806,0.042641,2
1,84.712442,1.352216,0.581354,0.028348,1.0,3000,"{'learning_rate': 1.0, 'n_estimators': 3000}",0.548673,0.642857,0.607143,0.589286,0.687500,0.615092,0.047232,3
2,70.148801,10.567941,0.501157,0.120712,1.5,3000,"{'learning_rate': 1.5, 'n_estimators': 3000}",0.601770,0.660714,0.607143,0.616071,0.696429,0.636425,0.036505,1


In [52]:
model_2 = clf.best_estimator_
model_2

AdaBoostClassifier(learning_rate=1.5, n_estimators=3000, random_state=42)

In [53]:
y_pred_2 = model_2.predict(x_val)
y_pred_2

array([-1, -1,  0, -1,  0,  0,  0, -1, -1,  0,  0, -1,  0, -1,  0,  0,  0,
        0,  1,  0,  0,  0, -1, -1,  0,  0, -1,  0,  0, -1, -1,  0,  0,  0,
       -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0, -1,  0,
       -1, -1, -1,  0, -1,  0,  0,  0,  1, -1, -1,  0,  0,  0,  1,  0, -1,
       -1,  0,  0, -1,  0,  0,  1,  0,  0, -1, -1, -1, -1, -1,  0,  0, -1,
        0, -1, -1,  0,  0, -1,  0, -1, -1,  0,  1, -1, -1,  1, -1, -1, -1,
        0, -1,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1, -1,  0, -1, -1,
        0, -1,  1,  0, -1,  0,  0, -1, -1, -1,  0, -1, -1, -1,  1, -1,  0,
       -1,  0,  0,  0,  0], dtype=int64)

In [54]:
print(classification_report(y_val, y_pred_2))

              precision    recall  f1-score   support

          -1       0.71      0.67      0.69        61
           0       0.61      0.76      0.68        59
           1       0.78      0.33      0.47        21

    accuracy                           0.66       141
   macro avg       0.70      0.59      0.61       141
weighted avg       0.68      0.66      0.65       141



# Test custom data

In [55]:
def classifier(model, text):
    embeddings_test = model_transform.encode(text)
    y_pred = model.predict(embeddings_test)
    df_results = pd.DataFrame({'Text' : text, 'label' : y_pred})

    return df_results

In [56]:
x = ['Bolsonaro é o melhor presidente possível', 'Lula sempre deu o seu melhor pelo país', 'Ciro coitado, não serve pra nada', 'Todos os candidatos são ruins', 'Gostei muito dessa proposta', 'Bolsonaro foi ao congresso', 'Essa eleição é uma piada mesmo', 'Simplesmente horrível política', 'Não consigo entender como tem gente que não gosta de política no Brasil', 'Estou muito contente com o meu canditato!', 'Não acho é nada sobre isso', 'Sério que você acha o lula bom? kkkkkkkkkkkkk', "Ele é um ladrão velho", "Eleições são uma perda de tempo mesmo", 'Qualquer candidato eleito vai continuar a mesma coisa', '5 dias pro Lula ganhar no primeiro turno em galera', 'O brasil já está desacreditado de política nos dias atuais', 'insuportável essa época de política', 'Vai deixar de gostar do Neymar por conta de posição política? Eu nunca gostei do Neymar!.']

# KNN

In [57]:
df_results = classifier(model_knn, x)

In [60]:
from IPython.display import HTML

HTML(df_results.to_html())

,Text,label
0,Bolsonaro é o melhor presidente possível,1
1,Lula sempre deu o seu melhor pelo país,-1
2,"Ciro coitado, não serve pra nada",-1
3,Todos os candidatos são ruins,-1
4,Gostei muito dessa proposta,1
5,Bolsonaro foi ao congresso,1
6,Essa eleição é uma piada mesmo,-1
7,Simplesmente horrível política,-1
8,Não consigo entender como tem gente que não gosta de política no Brasil,-1
9,Estou muito contente com o meu canditato!,1


In [ ]:
df_results.to_csv('../data/result.csv',index=False)